In [ ]:
using SequentialMonteCarlo
using Distributions
using DifferentialEquations
using DelimitedFiles
using Statistics
using Plots
using LinearAlgebra
using StatsBase
using StatsPlots

In [ ]:
y_full = readdlm("lvsmc/prey_pred2.csv", ',', Float64)
n_y = size(y_full,2) - 1 # exclude initial value
ts = 1.0:1.0:n_y # times steps for data

In [ ]:
# particle
mutable struct LVParamParticle
    logα::Float64 
    logβ::Float64 
    logγ::Float64
    logσ::Float64
    LVParamParticle() = new() # initialise empty
end

Base.vec(p::LVParamParticle) = [p.logα, p.logβ, p.logγ, p.logσ]
vecexp(p::LVParamParticle) = exp.(vec(p))

In [ ]:
# Prior on θ = log([α, β, γ, σ])
priors = product_distribution([Uniform(-2, 1),Uniform(-8,-1),Uniform(-2, 1),Uniform(-2, 3)])

function log_prior(logθ::Vector{Float64})
    # Prior distributions.
    return sum(logpdf.(priors, logθ))
end

In [ ]:
# Define Lotka-Volterra model.
function lotka_volterra(du, u, p, t)
    # Model parameters.
    α, β, γ = p
    # Current state.
    x, y = u

    # Evaluate differential equations.
    du[1] = (α - β * y) * x # prey
    du[2] = (β * x - γ) * y # predator

    return nothing
end


# Define initial-value problem.
u0 = y_full[:,1]
p_true = [0.5, 0.0025, 0.3, 1.]
tspan = (0.0, 50.0)
prob = ODEProblem(lotka_volterra, u0, tspan, p_true[1:3])


In [ ]:
y = y_full[:,2:end]
temps = 0.5 .^ (12:-1:0)
n = length(temps) - 1

function log_like(logθ::Vector{Float64})
    α, β, γ, σ =  exp.(logθ)
    # Simulate Lotka-Volterra model. 
    p = [α, β, γ]
    predicted = solve(prob, Tsit5(); p=p, saveat = ts, verbose=false, alg_hints = :stiff)

    if !SciMLBase.successful_retcode(predicted.retcode)
        #println(p)
        return -Inf
    end

    # Add up the log likelihood
    log_likelihood = 0.0
    for i in eachindex(predicted)
        like_dist = MvNormal(predicted[:,i], σ^2 * I)
        log_likelihood += logpdf(like_dist, y[:,i])
    end
    return log_likelihood
end

In [ ]:

function proposal(currθ::Vector{Float64})
    return MvNormal(currθ, 0.25^2 * I)
end

# Metropolis-Hastings Kernel with Random Walk proposal
# Posterior = prior(θ) * likelihood(θ)ᵝ
function mh(rng, currθ::Vector{Float64}, propθ::Vector{Float64}, β::Float64)
    lp_curr = logpdf(priors, currθ) + β * log_like(currθ)
    lp_prop = logpdf(priors, propθ) + β * log_like(propθ)
    if lp_prop - lp_curr > log(rand(rng))
        return propθ
    else
        return currθ
    end
end

In [ ]:
# mutation kernel
function M!(new::LVParamParticle, rng, t::Int64, old::LVParamParticle, ::Nothing)
    if t == 1
        logθ = rand(rng, priors)
    else
        # proposal
        propθ = rand(rng, proposal(vec(old)))
        logθ = mh(rng, vec(old), propθ, temps[t])
    end

    new.logα, new.logβ, new.logγ, new.logσ = logθ

end

In [ ]:

# potential function
function lG(t::Int64, particle::LVParamParticle, ::Nothing)
    β_incr = temps[t+1] - temps[t]
    return β_incr * log_like(vec(particle))
end

In [ ]:

N = 2^10        # number of particles
threads = 1     # number of threads
κ = 0.5         # relative ESS threshold
saveall = true  # save full trajectory 
model = SMCModel(M!, lG, n, LVParamParticle, Nothing)
smcio = SMCIO{model.particle, model.pScratch}(N, n, threads, saveall, κ)
smc!(model, smcio)
